# Import Libraries

In [1]:
import json
import requests
import pandas as pd
import os
import sys
import configparser
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load Nansen API Token
Nansen API Token: https://query.nansen.ai/users/me

In [2]:
config = configparser.RawConfigParser()
config.read('config.ini')
try:
    NANSEN_KEY = config['info']['nansen_token']
except KeyError:
    os.system("cls" if os.name == "nt" else "clear")
    print("Run setup.py first")
    sys.exit(1)

# Set ERC20 Token name and address

In [3]:
TOKEN_NAME = "INJUSDT" # TOKEN_NAME must be listed on Binance.
TOKEN_ADDRESS = "0xe28b3b32b6c345a34ff64674606124dd5aceca30" # TOKEN_ADDRESS must always start with "0x".

# Get data from Nansen API

In [4]:
NANSEN_URL = 'https://query.api.nansen.ai/v1/questions/api_tgm_top_transactions_7d'
NANSEN_PARAMS = {"token": TOKEN_ADDRESS}
headers = {'Accept': 'application/json','Content-Type': 'application/json','Authorization': NANSEN_KEY}
payload =  json.dumps({"params": NANSEN_PARAMS, "accept_stale": False})

try:
    r = requests.post(NANSEN_URL, headers=headers, data=payload)
    r.raise_for_status()
    nansen_df = pd.DataFrame.from_dict(r.json()['result_data'])
except (requests.HTTPError, KeyError):
    print('API call failed. Check your token or query.')
    sys.exit(1)

In [5]:
nansen_df[:13 + 1]

,from_address,time,to_address,token_address,transaction_hash,value
0,0xd55923b9bd1bc209c3470e946cc1105b30a84b31,{'seconds': 1698692567},0xf7dcfe0c18647ff66caa629439bbbe9a643c273b,0xe28b3b32b6c345a34ff64674606124dd5aceca30,0xa96d6c09eddcb291dcba69407aeb2e8006950c182ccd...,71212.765778
1,0xf7dcfe0c18647ff66caa629439bbbe9a643c273b,{'seconds': 1698693071},0xa9d1e08c7793af67e9d92fe308d5697fb81d3e43,0xe28b3b32b6c345a34ff64674606124dd5aceca30,0x3ba2dcfb37660f35e5f2bcd0884eafca0e0b14e04fe8...,71212.765778
2,0x85d17b1f3babb13352ed475059e2d1dd853760cc,{'seconds': 1698901607},0x32cb160edf4b7d4d1390d0255d6693efe5381f28,0xe28b3b32b6c345a34ff64674606124dd5aceca30,0x6316e09aace0b0959c35f7de24c1b462f7481e48bf3a...,71212.765778
3,0x32cb160edf4b7d4d1390d0255d6693efe5381f28,{'seconds': 1698901871},0xa9d1e08c7793af67e9d92fe308d5697fb81d3e43,0xe28b3b32b6c345a34ff64674606124dd5aceca30,0x67afd54ab17b723c28b9acda457e47ffb4e3d96d2591...,71212.765778
4,0x3e12fae946d773b632db1ed03d295e7fe202a046,{'seconds': 1699000235},0xb01629568c9af7bd6080d08cec27134917452a6c,0xe28b3b32b6c345a34ff64674606124dd5aceca30,0x80a8f885f03721ce89b289cef2c9ab35fdc463cf450e...,71179.331686
5,0xb01629568c9af7bd6080d08cec27134917452a6c,{'seconds': 1699000571},0xa9d1e08c7793af67e9d92fe308d5697fb81d3e43,0xe28b3b32b6c345a34ff64674606124dd5aceca30,0x997210b8e0940f0362cf4c6bb4b4cd50dc07b0b6d94c...,71179.331686
6,0x9ff41cba4a131350b3e3056c315dca6e5a63d9e4,{'seconds': 1698505811},0xb514f4a77ce08d8694b242710920e8fa66dae1f5,0xe28b3b32b6c345a34ff64674606124dd5aceca30,0xcc63d58d784c2b2a14dc90773858f82e8016beb03173...,51458.100000
7,0xb514f4a77ce08d8694b242710920e8fa66dae1f5,{'seconds': 1698595751},0xa7665b0ae33b3336db20df4caa42fa377067b4c0,0xe28b3b32b6c345a34ff64674606124dd5aceca30,0xab0ac4fa01e8cdacdeeb55f8b4ee3060141ac4d26514...,44458.100000
8,0xa7665b0ae33b3336db20df4caa42fa377067b4c0,{'seconds': 1698743531},0x3612547276b199cc044ea0a6d0a974af56776c14,0xe28b3b32b6c345a34ff64674606124dd5aceca30,0xd87ef9bd1cb6a064fc11e541f838a335aefcb3fd9d7c...,42158.100000
9,0xd5417e96dd04363c675e41ee6f30bf788412c719,{'seconds': 1698749003},0x66e422eaa2173d3aefd130239b55b2d3a2216f94,0xe28b3b32b6c345a34ff64674606124dd5aceca30,0x63e1d3672b2da1ce061be0dde4148c3c777ec8dc6156...,40958.649199


# Fetch address label from Arkham

In [6]:
all_unique_addresses = set(nansen_df[['from_address', 'to_address', 'token_address']].values.ravel())

In [7]:
def fetch_label(UNIQUE_ADDR):
    ARKHAM_URL = f"https://api.arkhamintelligence.com/intelligence/address/{UNIQUE_ADDR}?chain=ethereum"
    
    try:
        response = requests.get(ARKHAM_URL).json()
        arkham_entity_name = response.get('arkhamEntity', {}).get('name')
        arkham_label_name = response.get('arkhamLabel', {}).get('name')
        populated_tags_list = response.get('populatedTags', [])
        populated_tags = populated_tags_list[0].get('label') if populated_tags_list else None
        
        if arkham_entity_name and arkham_label_name:
            TOKEN_LABEL = f"{arkham_entity_name}: {arkham_label_name}"
        elif arkham_entity_name:
            TOKEN_LABEL = arkham_entity_name
        elif arkham_label_name:
            TOKEN_LABEL = arkham_label_name
        elif populated_tags:
            TOKEN_LABEL = populated_tags
        else:
            TOKEN_LABEL = "Unknown"
    except Exception as e:
        TOKEN_LABEL = "Unknown"

    return f"{TOKEN_LABEL} [{UNIQUE_ADDR[:8]}]"

In [8]:
address_to_label = {}

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = {executor.submit(fetch_label, addr): addr for addr in all_unique_addresses}

    for future in as_completed(futures):
        addr = futures[future]
        address_to_label[addr] = future.result()

        time.sleep(0.1)

# Apply label to dataframe

In [9]:
target_cols = ['from_address', 'to_address', 'token_address']
nansen_df[target_cols] = nansen_df[target_cols].applymap(lambda x: address_to_label.get(x, x))

In [10]:
nansen_df['time'] = pd.to_datetime(nansen_df['time'].str['seconds'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
nansen_df['value'] = nansen_df['value'].astype(float).map('{:,.0f}'.format)
nansen_df = nansen_df[['token_address', 'from_address', 'to_address', 'value', 'time']]

In [11]:
nansen_df[:13 + 1]

,token_address,from_address,to_address,value,time
0,Injective Protocol: Injective Token (INJ) [0xe...,Unknown [0xd55923],Coinbase Deposit [0xf7dcfe],"71,213",2023-10-30 19:02:47
1,Injective Protocol: Injective Token (INJ) [0xe...,Coinbase Deposit [0xf7dcfe],Coinbase [0xa9d1e0],"71,213",2023-10-30 19:11:11
2,Injective Protocol: Injective Token (INJ) [0xe...,Unknown [0x85d17b],Coinbase Deposit [0x32cb16],"71,213",2023-11-02 05:06:47
3,Injective Protocol: Injective Token (INJ) [0xe...,Coinbase Deposit [0x32cb16],Coinbase [0xa9d1e0],"71,213",2023-11-02 05:11:11
4,Injective Protocol: Injective Token (INJ) [0xe...,Unknown [0x3e12fa],Unknown [0xb01629],"71,179",2023-11-03 08:30:35
5,Injective Protocol: Injective Token (INJ) [0xe...,Unknown [0xb01629],Coinbase [0xa9d1e0],"71,179",2023-11-03 08:36:11
6,Injective Protocol: Injective Token (INJ) [0xe...,Coinbase Prime Custody [0x9ff41c],Coinbase Prime Custody [0xb514f4],"51,458",2023-10-28 15:10:11
7,Injective Protocol: Injective Token (INJ) [0xe...,Coinbase Prime Custody [0xb514f4],Coinbase Prime Custody [0xa7665b],"44,458",2023-10-29 16:09:11
8,Injective Protocol: Injective Token (INJ) [0xe...,Coinbase Prime Custody [0xa7665b],Coinbase Prime Custody [0x361254],"42,158",2023-10-31 09:12:11
9,Injective Protocol: Injective Token (INJ) [0xe...,Early EZ Holder [0xd5417e],Unknown [0x66e422],"40,959",2023-10-31 10:43:23


# Load Binance data for integration with Nansen data

In [12]:
from market_data import get_data
PRICE_DATA = get_data(ticker=TOKEN_NAME, days=10, ts="5m")

# Match the time format for plotting

In [13]:
PRICE_DATA.index = pd.to_datetime(PRICE_DATA.index)
nansen_df['time'] = pd.to_datetime(nansen_df['time'])

# Import libraries for plotting

In [14]:
import plotly.graph_objects as go
import numpy as np
from scipy.spatial import cKDTree
from datetime import timedelta

# Plotting price and transaction value

In [15]:
nansen_df = nansen_df[:13 + 1]

In [16]:
fig = go.Figure()

# Add price line
fig.add_trace(go.Scatter(x=PRICE_DATA.index, y=PRICE_DATA['close'], mode='lines', name='Price'))

label_positions = []
kdtree = None
point_x, point_y, label_x, label_y = [], [], [], []

# Calculate y-axis range for label positioning
min_price, max_price = min(PRICE_DATA['close']), max(PRICE_DATA['close'])
buffer = (max_price - min_price) * 0.1

chart_labels = []
legend_dict = {}

for idx, (i, row) in enumerate(nansen_df.iterrows()):
    trade_time = row['time']
    trade_value = row['value']
    
    # Address info
    from_address = row['from_address'].split(' ')[0]
    to_address = row['to_address'].split(' ')[0]
    
    # String for legend and chart label
    legend_str = f"{from_address} -> {to_address}"
    chart_label = f"[{chr(65 + idx)}] {trade_value}"
    
    # Update legend dictionary
    legend_dict[chr(65 + idx)] = legend_str
    
    closest_time_index = np.abs(PRICE_DATA.index.to_pydatetime() - pd.Timestamp(trade_time).to_pydatetime()).argmin()
    closest_time = PRICE_DATA.index[closest_time_index]
    trade_price = PRICE_DATA.loc[closest_time, 'close']

    # Store actual points
    point_x.append(closest_time)
    point_y.append(trade_price)

    # Generate candidate label positions
    candidate_positions = [(trade_price + i * buffer, 'up') for i in range(1, 6)] + \
                          [(trade_price - i * buffer, 'down') for i in range(1, 6)]

    # Use k-d tree to find optimal label position
    if kdtree:
        distances, _ = kdtree.query(np.array([pos[0] for pos in candidate_positions]).reshape(-1, 1))
    else:
        distances = [float('inf')] * len(candidate_positions)

    optimal_position, _ = candidate_positions[np.argmax(distances)]
    label_positions.append([optimal_position])
    kdtree = cKDTree(np.array(label_positions).reshape(-1, 1))

    label_x.append(closest_time)
    label_y.append(optimal_position)
    chart_labels.append(chart_label)

# Add points and labels
fig.add_trace(go.Scatter(x=point_x, y=point_y, mode='markers', marker=dict(color='red', size=5), name='TX Points', legendgroup='group'))
fig.add_trace(go.Scatter(x=label_x, y=label_y, mode='text', text=chart_labels, name='TX Values', textfont=dict(color='black', size=12)))


# Add lines connecting labels to points
for px, py, lx, ly in zip(point_x, point_y, label_x, label_y):
    fig.add_shape(type="line", x0=px, y0=py, x1=lx, y1=ly, line=dict(color="Gray", width=0.5))

# Update layout with custom legend
legend_text = "<br>".join([f"{key}: {value}" for key, value in legend_dict.items()])
one_day_later = max(PRICE_DATA.index) + timedelta(days=1)
fig.update_layout(
    title=f"{TOKEN_NAME} 5m with Top Transactions",
    xaxis=dict(
        range=[min(PRICE_DATA.index), one_day_later]
    ),
    xaxis_title='Time',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    showlegend=False,
    annotations=[dict(
        x=1.05,
        y=1,
        align="left",
        valign="top",
        text=legend_text,
        showarrow=False,
        xref="paper",
        yref="paper",
        xanchor="left",
        yanchor="top",
        width=300
    )],
    height=550,
    margin=dict(
        r=250,
        b=150
    )
)

fig.show()